# Change constraints 

In stochastic programs with chance constraints the goal is to make an optimal decision prior to the realization of random data while allowing the constraints (or some of them) to be violated with a certain probability. We note that chance constraints are also called probabilistic constraints.

## General mathematical formulation of a stochastic programm with change constraints

Let $x$ denote the decision variable, $c$ the coefficients of the objective function, $A$ a random matrix representing the left-hand side of the constraints and $b$ a random vector representing the right-hand side of the constraints.

$$
\begin{array}{ll}
\min & c\cdot x \\
s.t. & P(Ax \leq b) \geq p \\
     & x \geq 0
\end{array}
$$

One way to solve such a stochastic problem is to translate it into an equivalent mixed-integer problem (MIP).
For $S$ a given set of scenarios, we introduce binary variables $y_k$ for each scenario $k\in S$, which takes the value 1 if and only if the corresponding constraint is satisfied in scenario $k$. 

The equivalent scenario based formulation using the Big-M method of above stochatic programm with change constraints is the following:

$$
\begin{array}{ll}
\min & c \cdot x \\
s.t. & A_kx \leq b_k + M_k(1-y_k) \\
     & \sum_{k\in S} y_k \geq p\times |S| \\
     & x \geq 0, \; y\in\{0,1\}^{|S|}
\end{array}
$$

### Single chance constraints

- TBD
- only one constraint has to be satisfied with some probability

### Multiple chance consttraints

- TBD
- more than on constraint has to be satisfied with some probability
- joint chance constraints
- individual chance constraints

# An ambulance service

The problem consists of locating an emergency unit (such as anambulance) in a region.

sources:
- {cite}`birge2011introduction`
- {cite}`toregas1971location`

## math formulation

Consider the following covering location problem. Let
- $j = \{1, . . . , n\}$ be the potential locations for a site.
- $x_j$ be a binary variable with $x_j = 1$ if site $j$ is open and $0$ otherwise.
- $c_j$ be the investment cost,
- $i = \{1, . . . , m\}$ be the villages. 
- $t_{i,j}$ the distance between $i$ and $j$ 

village $i$ is served if there exists an open site within distance $t_i$ .
We define $N_i = \{ j | t_{i,j} < t_i\}$ as the set of eligible sites for village $i$ . 

The deterministic covering problem is

$$
\begin{array}{lllr}
\min & \sum_{j=1}^n c_jx_j & & \\
s.t. & \sum_{j\\in N_j} x_j\geq 1&, i=1,\ldots,m &(c1)\\
 & x_j\in\{0,1\} &, j= 1,\ldots,n &
\end{array}
$$

In [ ]:
#export
import pyomo.environ as pyo
import pandas as pd
import matplotlib.pyplot as plt
import random
from math import ceil, log
from matplotlib.pyplot import figure

figure(figsize=(20, 16), dpi=80)
random.seed(110)

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# data
villages = ['a','b','c','d','e','f','g']
locations = [1,2,3,4,5,6,7,8,9,10,11]

In [ ]:
# helper
def x_l(j):
    return location.loc[location.location==j, 'x'].values[0]
def y_l(j):
    return location.loc[location.location==j, 'y'].values[0]
def x_c(i):
    return village.loc[village['village'] == i, 'x'].values[0]
def y_c(i):
    return village.loc[village['village'] == i, 'y'].values[0]

def dist(x,y):
    return (x**2 + y**2)**(1./2.)

In [ ]:
cost = {j: random.uniform(1,5) for j in locations}
t = {i: random.uniform(1,9) for i in villages}

location = pd.DataFrame({
    'location': locations,
    'x': [random.uniform(0,10) for j in locations],
    'y': [random.uniform(0,10) for j in locations]    
})

village = pd.DataFrame({
    'village': villages,
    'x': [random.uniform(0,10) for j in villages],
    'y': [random.uniform(0,10) for j in villages]    
})


distance = pd.DataFrame({'village': villages})
for j in locations:
    distance[j] = [dist(x_l(j) - x_c(i), y_l(j) - y_c(i)) for i in villages]
distance.set_index('village', inplace =True)
    
def N_eligible_set(i, distance=distance, t = t):
    return [j for j in locations if distance.loc[i,j] < t[i]]

NameError: name 'locations' is not defined

In [ ]:
#export
def generate_data(villages, locations):
    """generate random data an returns list: 
    1. location data: coordinates and opening costs
    2. village data: coordinates and service range
    3. distance matrix village locations"""
    # helper functions
    def x_l(j):
        return location.loc[location.location==j, 'x'].values[0]
    def y_l(j):
        return location.loc[location.location==j, 'y'].values[0]
    def x_c(i):
        return village.loc[village['village'] == i, 'x'].values[0]
    def y_c(i):
        return village.loc[village['village'] == i, 'y'].values[0]
    def dist(x,y):
        return (x**2 + y**2)**(1./2.)
    # generate random data
    #cost = {j: random.uniform(1,5) for j in locations}
    #t = {i: random.uniform(1,9) for i in villages}
    
    location = pd.DataFrame({
        'location': locations,
        'x': [random.uniform(0,10) for j in locations],
        'y': [random.uniform(0,10) for j in locations],
        'cost': [random.uniform(1,5) for j in locations]
    })
    village = pd.DataFrame({
        'village': villages,
        'x': [random.uniform(0,10) for j in villages],
        'y': [random.uniform(0,10) for j in villages],
        'service_range': [random.uniform(1,10) for i in villages]
    })
    
    distance = pd.DataFrame({'village': villages})
    for j in locations:
        distance[j] = [dist(x_l(j) - x_c(i), y_l(j) - y_c(i)) for i in villages]
    distance.set_index('village', inplace =True)
    
    return [location, village, distance]
    
#def N_eligible_set(i, distance, t):
#    return [j for j in locations if distance.loc[i,j] < t[i]]

def N_eligible_set(i, distance, village, location):
    """returns eliglble sets per village"""
    locations = location.location
    t_i = village.loc[village.village == i, 'service_range'].values[0] 
    return [j for j in locations if distance.loc[i,j] < t_i]

In [ ]:
# generate data
location, village, distance = generate_data(villages, locations)

In [ ]:
# check if for each village there is at least one location in range
# empty list means data does not provide a feasible solution
[N_eligible_set(i, distance, village, location) for i in villages]

In [ ]:
#export
def ambulance_deterministic(village, location, distance):
    m = pyo.ConcreteModel()
    
    # sets
    m.I = pyo.Set(initialize = list(village.village), doc = 'villages')
    m.J = pyo.Set(initialize = list(location.location), doc = 'locations')
    
    # parameter
    @m.Param(m.J, doc = 'lodaction costs')
    def c(m,j):
        return location.loc[location.location == j, 'cost'].values[0]
    
    # vars
    m.x = pyo.Var(m.J, domain = pyo.Binary, doc = '1 iff location j is open')
    
    # objective
    m.obj = pyo.Objective(expr = sum(m.c[j] * m.x[j] for j in m.J), sense = pyo.minimize, doc = 'overall costs')
    
    # constraints
    @m.Constraint(m.I)
    def c1(m,i):
        return sum(m.x[j] for j in N_eligible_set(i, distance, village, location)) >= 1
    
    # choose solver and solve
    solver = pyo.SolverFactory('glpk')
    solver.solve(m)
    
    return m

In [ ]:
m = ambulance_deterministic(village, location, distance)

In [ ]:
#export
def plot_solultion(location, village, demand = None):
    # helper functions
    def x_l(j):
        return location.loc[location.location==j, 'x'].values[0]
    def y_l(j):
        return location.loc[location.location==j, 'y'].values[0]
    def x_c(i):
        return village.loc[village['village'] == i, 'x'].values[0]
    def y_c(i):
        return village.loc[village['village'] == i, 'y'].values[0]
    def dist(x,y):
        return (x**2 + y**2)**(1./2.)
    # plot
    x_loc = location.x
    y_loc = location.y
    x_cli = village.x
    y_cli = village.y
    # size and color:
    fig, ax = plt.subplots()
    ax.scatter(x_loc, y_loc, c='b', label = 'Locations')
    ax.scatter(x_cli, y_cli, c='r', label = 'villages')
    
    if demand == None:
        for i in village.village:
            x = village.loc[village['village']==i,'x'].values[0]
            y = village.loc[village['village']==i,'y'].values[0]
            t_i = village.loc[village.village == i, 'service_range'].values[0] 
            circle = plt.Circle((x, y), t_i, color='r', alpha = 0.03)
            ax.add_patch(circle)
    else:
        for i in demand.keys():
            ax.annotate(str(round(demand[i],2)), (x_c(i), y_c(i)))
            
    # add labels of location and village
    for i in location.location:
        ax.annotate(str(i), (x_l(i), y_l(i)))
    
        
    plt.legend(loc="lower left")
    
    return plt.show()

In [ ]:
#export
def extract_solution(m):
    """extracts solution locacation + overall costs"""
    solution_loc = [j for j in m.J if pyo.value(m.x[j]) == 1]
    solution_cost = pyo.value(m.obj)
    return (solution_loc, [solution_cost])

# Ambulance example with chance constraints

We recall our covering location problem:

We have to decide which of $n$ possible locations $j\in\{1,\ldots,n\}$ we use as as an ambulance service 


If a new emergency call arrives from region, the emergency unit may be busy and serving another call. This may happen at location $j$ with probability $p$. For simplicity we assume that this probability is the same at all locations.

This means the requirement, that each village $i$ has a location in its range $t_i$, needs to be replaced by the requirement that the probability $P(\text{at least one emergency unit from in range is available})\geq \alpha$, where $\alpha$ denotes some confidence level for example $90\%$ or $95\%$.

As the probability that none emergency unit is available in range of village $i$ is $p^{\sum_{j\in N_i} x_j}$, the probabilistic constraint is 

$$
1 - p^{\sum_{j\in N_i} x_j} \geq \alpha
$$

This is non linear with respect to our decision variables, we take a logarithm (as we like to do linear programming):

$$
\begin{array}{ll}
& \log(p) \cdot \sum_{j\in N_i} x_j \leq \log(1-\alpha)\\
& \sum_{j\in N_i} x_j \geq \lceil \frac{\log(1-\alpha)}{\log(p)}\rceil
\end{array}
$$

where for a real number $x$ the symbol $\lceil x \rceil$ denotes the smallest integer greater or equal to $x$.

Putting all together we derive the following:

$$
\begin{array}{lllc}
\min & \sum_{j=1}^n c_j x_j & & \\
s.t. & \sum_{j\in N_i} x_j \geq \lceil \frac{\log(1-\alpha)}{\log(p)}\rceil, & i\in\{1,\ldots,m\} & (c1)\\
     & x_j\in \{1,0\}, & j\in\{1,\ldots,n\} &
\end{array}
$$